In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('taxi-data-groupby-test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 14:32:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 14:32:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### green data

In [32]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [34]:
df_green.createOrReplaceTempView('green')


In [36]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [38]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-06 07:00:00| 241|            169.16|             6|
|2020-01-19 19:00:00|  41| 457.2400000000003|            43|
|2020-01-23 23:00:00| 129| 304.1300000000001|            22|
|2020-01-28 15:00:00| 129|            208.23|             9|
|2020-01-02 08:00:00|  74|1187.4199999999987|            96|
|2020-01-06 16:00:00| 129|250.42000000000002|            15|
|2020-01-07 10:00:00| 171|             38.98|             2|
|2020-01-09 15:00:00| 260| 242.6800000000001|            19|
|2020-01-16 20:00:00|  82| 424.1900000000001|            27|
|2020-01-21 16:00:00|  33| 917.3599999999994|            38|
|2020-01-08 17:00:00| 140|            313.81|             8|
|2020-01-22 08:00:00| 244| 285.3999999999999|            14|
|2020-01-30 14:00:00|  74| 738.6799999999996|            49|
|2020-01-03 07:00:00|  7

In [40]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

24/03/02 22:27:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 22:27:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


#### yellow data

In [42]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [44]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [46]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-25 01:00:00| 186|2287.5599999999986|           128|
|2020-01-24 07:00:00| 186| 7247.550000000012|           483|
|2020-01-04 18:00:00| 161| 7807.060000000018|           487|
|2020-01-03 12:00:00| 186| 7301.100000000017|           418|
|2020-01-22 19:00:00| 140|           4461.66|           263|
|2020-01-21 13:00:00| 162| 8327.450000000013|           491|
|2020-01-06 20:00:00| 234|5267.8600000000015|           333|
|2020-01-23 12:00:00| 233| 3545.589999999998|           184|
|2020-01-15 13:00:00| 239| 4100.529999999999|           289|
|2020-01-17 08:00:00| 170| 7410.990000000011|           475|
|2020-01-22 21:00:00|  79| 6247.700000000006|           358|
|2020-01-15 15:00:00| 239| 5217.580000000002|           344|
|2020-01-11 18:00:00| 236| 9088.440000000017|           612|
|2020-01-12 17:00:00| 23

In [48]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/02 22:27:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 22:27:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### join two large tables

In [50]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [52]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [54]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [56]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000001|                   57|
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|  56|             99.69|                   3|              18.1|                    2|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|199.49000000000004|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  74|317.09000000000015|                  24| 586.2100000000002|                   47|
|2020-01-01 00:00:00|  88|  

In [58]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

24/03/02 22:54:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### join large table with small table(zones)

In [60]:
df_zones = spark.read.parquet('zones/')

In [62]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [64]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [66]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000001|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|        10|   Queens|        Baisley Park|   Boro Zone|
|2020-01-01 00:00:00|  56|             99.69|                   3|              18.1|                    2|        56|   Queens|              Corona|   Boro Zone|
|2020-01-01 00:00:00| 

In [68]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|              NULL|                NULL|1004.3000000000001|                   57|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              NULL|                NULL|             42.41|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|             99.69|                   3|              18.1|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|            526.71|                  17|            146.64|                    3| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|199.49000000000004|                  10|            409.35|                   19| Brooklyn|   Bor

In [70]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

24/03/02 22:55:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
